# Pattern Recognition, Homework 2

* **Author** Haonan Chen
* **Class** 32
* **Student No.** 2013011449
* **Date** 2016/10/22
* **Language** [Julia](http://julialang.org/)

## Read training and testing data

Using `Pkg.add(package)` to install packages.

In [1]:
using DataFrames
using Distributions
using Distances
using MultivariateStats
using LIBSVM

In [2]:
df_train = readtable("dataset3.txt", separator = '\t')
df_test = readtable("dataset4.txt", separator = '\t')
names!(df_train, cat(1, [Symbol("F$i") for i in 1:10],[Symbol("Sex")]))  # assign a name to each column
names!(df_test,  cat(1, [Symbol("F$i") for i in 1:10],[Symbol("Sex")]))
df_test[df_test[:,:Sex].=="f",:Sex]="F"
by(df_train, :Sex) do df
  DataFrame(
    number = length(df[:Sex]),
  )
end

,Sex,number
1,F,468
2,M,485


In [3]:
by(df_test, :Sex) do df
  DataFrame(
    number = length(df[:Sex]),
  )
end

,Sex,number
1,F,77
2,M,250


In [4]:
df_train_two_F = df_train[df_train[:,:Sex].=="F",[:F3,:F5]]
df_train_two_M = df_train[df_train[:,:Sex].=="M",[:F3,:F5]]
df_train_two_F_s = df_train_two_F[sample(1:size(df_train_two_F, 1), 10, replace = false), :]

,F3,F5
1,54.0,165.0
2,50.0,157.0
3,49.0,172.0
4,53.0,165.0
5,51.0,172.0
6,48.0,150.0
7,42.5,160.0
8,58.0,165.0
9,50.0,160.0
10,67.0,160.0


In [5]:
df_train_two_M_s = df_train_two_M[sample(1:size(df_train_two_M, 1), 10, replace = false), :]

,F3,F5
1,80.0,176.0
2,73.0,176.0
3,64.0,179.0
4,85.0,171.0
5,70.0,173.0
6,70.0,172.0
7,72.0,169.0
8,68.0,180.0
9,77.0,175.0
10,72.0,170.0


In [6]:
df_train_ten_F = df_train[df_train[:,:Sex].=="F", 1:10]
df_train_ten_M = df_train[df_train[:,:Sex].=="M", 1:10]
df_train_ten_F_s = df_train_ten_F[sample(1:size(df_train_ten_F, 1), 11, replace = false), :]

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10
1,282.24,33.64,58.0,20.55,168.0,162.0,62.0,166.82,163.64,172.1
2,299.29,40.96,64.0,21.38,173.0,165.0,55.0,173.37,167.93,193.36
3,256.0,26.01,51.0,19.92,160.0,179.0,63.0,154.33,175.41,186.56
4,265.69,28.09,53.0,19.95,163.0,175.0,56.0,185.55,173.46,169.35
5,256.0,24.01,49.0,19.14,160.0,182.0,67.0,173.25,171.56,172.51
6,256.0,21.16,46.0,17.97,160.0,179.0,75.0,168.29,166.21,194.84
7,256.0,25.0,50.0,19.53,160.0,160.0,50.0,178.74,167.99,182.98
8,265.69,30.25,55.0,20.7,163.0,172.0,64.0,184.24,169.06,173.31
9,256.0,30.25,55.0,21.48,160.0,156.0,40.0,168.4,173.58,189.7
10,259.21,29.16,54.0,20.83,161.0,175.0,60.0,162.09,176.58,174.42


In [7]:
df_train_ten_M_s = df_train_ten_M[sample(1:size(df_train_two_M, 1), 11, replace = false), :]

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10
1,299.29,36.0,60.0,20.05,173.0,178.0,70.0,162.95,171.17,149.87
2,338.56,60.84,78.0,23.04,184.0,170.0,72.0,175.43,166.52,196.14
3,316.84,42.25,65.0,20.52,178.0,163.0,65.0,164.61,178.37,155.38
4,309.76,43.56,66.0,21.31,176.0,166.0,52.0,153.7,170.49,156.21
5,316.84,49.0,70.0,22.09,178.0,180.0,49.0,166.78,178.23,190.02
6,299.29,56.25,75.0,25.06,173.0,160.0,62.0,181.74,179.88,186.35
7,327.61,39.69,63.0,19.23,181.0,180.0,59.0,162.84,168.47,197.27
8,316.84,51.84,72.0,22.72,178.0,182.0,47.0,183.09,166.97,150.21
9,327.61,42.25,65.0,19.84,181.0,172.0,70.0,175.91,165.59,155.46
10,309.76,39.69,63.0,20.34,176.0,180.0,80.0,171.34,164.25,174.61


## Sex Classification with Bayes Classifier

Follow this procedure:

* Fit Normal Distribution in each class in the training dataa to get parameters $\mu_i$ and $\Sigma_i$.
* Decision boundary can be determined by
$$
    -\frac12[(x-\mu_i)^\mathbf{T}\Sigma_i^{-1}(x-\mu_i)-(x-\mu_j)^\mathbf{T}\Sigma_j^{-1}(x-\mu_j)]-\frac12\ln \frac{|\Sigma_i|}{|\Sigma_j|}+\ln\frac{P(\omega_i)}{P(\omega_j)}=0
$$
* Then we can make prediction on test set.

Note that we are assuming the priori probability $P(\omega_i)=P(\omega_j)=0.5$, so the last item in the formular above can be ignored.

### Using all samples

In [8]:
# Distribution for female (female first)
norm_two_F = fit(MvNormal, Array(df_train_two_F)')

FullNormal(
dim: 2
μ: [51.7457,163.632]
Σ: [39.2591 16.5807; 16.5807 30.053]
)


In [9]:
# Distribution for male
norm_two_M = fit(MvNormal, Array(df_train_two_M)')

FullNormal(
dim: 2
μ: [68.0334,176.41]
Σ: [85.2565 23.3637; 23.3637 32.5048]
)


In [10]:
maha_two_M = pairwise(SqMahalanobis(norm_two_M.Σ.mat), Array(df_test[:,[:F3, :F5]])', norm_two_M.μ'')
maha_two_F = pairwise(SqMahalanobis(norm_two_F.Σ.mat), Array(df_test[:,[:F3, :F5]])', norm_two_F.μ'')
predict_two = (maha_two_M - maha_two_F + log(det(norm_two_M.Σ.mat)/det(norm_two_F.Σ.mat))) .< 0  # true denotes "M"
error_two = sum(Array(df_test[:, :Sex] .== "M").!=predict_two)/length(predict_two)  # error rate

0.14067278287461774

In [11]:
norm_ten_F = fit(MvNormal, Array(df_train_ten_F)')  # Distribution for female
norm_ten_M = fit(MvNormal, Array(df_train_ten_M)')  # Distribution for male
maha_ten_M = pairwise(SqMahalanobis(norm_ten_M.Σ.mat), Array(df_test[:,1:10])', norm_ten_M.μ'')
maha_ten_F = pairwise(SqMahalanobis(norm_ten_F.Σ.mat), Array(df_test[:,1:10])', norm_ten_F.μ'')
predict_ten = (maha_ten_M - maha_ten_F + log(det(norm_ten_M.Σ.mat)/det(norm_ten_F.Σ.mat))) .< 0  # true denotes "M"
error_ten = sum(Array(df_test[:, :Sex] .== "M").!=predict_ten)/length(predict_ten)  # error rate

0.16819571865443425

### Randomly pick 10 samples

In [12]:
norm_two_F_s = fit(MvNormal, Array(df_train_two_F_s)')  # Distribution for female
norm_two_M_s = fit(MvNormal, Array(df_train_two_M_s)')  # Distribution for male
maha_two_M_s = pairwise(SqMahalanobis(norm_two_M_s.Σ.mat), Array(df_test[:,[:F3, :F5]])', norm_two_M_s.μ'')
maha_two_F_s = pairwise(SqMahalanobis(norm_two_F_s.Σ.mat), Array(df_test[:,[:F3, :F5]])', norm_two_F_s.μ'')
predict_two_s = (maha_two_M_s - maha_two_F_s + log(det(norm_two_M_s.Σ.mat)/det(norm_two_F_s.Σ.mat))) .< 0  # true denotes "M"
error_two_s = sum(Array(df_test[:, :Sex] .== "M").!=predict_two_s)/length(predict_two_s)  # error rate

0.13149847094801223

In [13]:
norm_ten_F_s = fit(MvNormal, Array(df_train_ten_F_s)')  # Distribution for female
norm_ten_M_s = fit(MvNormal, Array(df_train_ten_M_s)')  # Distribution for male
maha_ten_M_s = pairwise(SqMahalanobis(norm_ten_M_s.Σ.mat), Array(df_test[:,1:10])', norm_ten_M_s.μ'')
maha_ten_F_s = pairwise(SqMahalanobis(norm_ten_F_s.Σ.mat), Array(df_test[:,1:10])', norm_ten_F_s.μ'')
predict_ten_s = (maha_ten_M_s - maha_ten_F_s + log(det(norm_ten_M_s.Σ.mat)/det(norm_ten_F_s.Σ.mat))) .< 0  # true denotes "M"
error_ten_s = sum(Array(df_test[:, :Sex] .== "M").!=predict_ten_s)/length(predict_ten_s)  # error rate

0.14984709480122324

## Sex Classification with LDA

For LDA, I use [MultivariateStats](https://github.com/JuliaStats/MultivariateStats.jl). See the document for more details.

A linear discriminant functional can be written as:
$$
f(\mathbf{x}) = \mathbf{w}^T \mathbf{x} + b
$$
Suppose the samples in the positive and negative classes respectively with means:  $\boldsymbol{\mu}_p$ and $\boldsymbol{\mu}_n$ , and within-class scatter matrixes  $\mathbf{S}_p$ and  $\mathbf{S}_n$. Then based on *Fisher's Linear Discriminant Criteria*, the optimal projection direction can be expressed as:
$$
\mathbf{w} = \alpha \cdot (\mathbf{S}_p + \mathbf{S}_n)^{-1} (\boldsymbol{\mu}_p - \boldsymbol{\mu}_n)
$$
Here $\alpha$ is an arbitrary non-negative coefficient.
The threshold $b$ can be obtained by:
$$
b = -\frac12(\boldsymbol{\mu}_p + \boldsymbol{\mu}_n)(\mathbf{S}_p + \mathbf{S}_n)^{-1} (\boldsymbol{\mu}_p - \boldsymbol{\mu}_n)-\ln\frac{P(\omega_p)}{P(\omega_n)}
$$
Again we assume $P(\omega_p)=P(\omega_n)=0.5$, then the last item can be ignored.

### Using all samples

In [14]:
lda_model_two = fit(LinearDiscriminant, Array(df_train_two_M)', Array(df_train_two_F)')

MultivariateStats.LinearDiscriminant{Float64}([0.0497863,0.0930593],-18.803757960270772)

In [20]:
error_lda_two = sum(Array(df_test[:, :Sex] .== "M") .!= predict(lda_model_two, Matrix(df_test[:, [:F3, :F5]])'))/size(df_test, 1)  # error rate

0.1162079510703364

In [21]:
lda_model_ten = fit(LinearDiscriminant, Array(df_train_ten_M)', Array(df_train_ten_F)')
error_lda_ten = sum(Array(df_test[:, :Sex] .== "M") .!= predict(lda_model_ten, Matrix(df_test[:, 1:10])'))/size(df_test, 1)  # error rate

0.10091743119266056

### Randomly pick 10 samples

In [22]:
lda_model_two_s = fit(LinearDiscriminant, Array(df_train_two_M_s)', Array(df_train_two_F_s)')
error_lda_two_s = sum(Array(df_test[:, :Sex] .== "M") .!= predict(lda_model_two_s, Matrix(df_test[:, [:F3, :F5]])'))/size(df_test, 1)

0.13761467889908258

In [24]:
lda_model_ten_s = fit(LinearDiscriminant, Array(df_train_ten_M_s)', Array(df_train_ten_F_s)')
error_lda_ten_s = sum(Array(df_test[:, :Sex] .== "M") .!= predict(lda_model_ten_s, Matrix(df_test[:, 1:10])'))/size(df_test, 1)

0.12232415902140673

## Sex Classification with Linear SVM

### Using all samples

### Randomly pick 10 samples

## Sex Classification with MLP

### Using all samples

### Randomly pick 10 samples